In [13]:
from transformers import pipeline # 30 seconds to load first time

In [5]:
import pandas as pd

Import your csv of reviews

In [15]:
df = pd.read_csv("resort_reviews.csv")

print(df.shape)     # rows, columns


(270, 5)


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       270 non-null    int64  
 1   resort          270 non-null    object 
 2   full_day_price  270 non-null    float64
 3   avg_rating      270 non-null    float64
 4   review          270 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 10.7+ KB


In [17]:
#Create new column called text_length
df['text_length'] = df['review'].apply(len)

In [18]:
df.describe()

,review_id,full_day_price,avg_rating,text_length
count,270.000000,270.000000,270.000000,270.000000
mean,135.500000,115.342593,4.242593,426.070370
std,78.086491,61.129686,0.581819,426.651393
min,1.000000,67.000000,1.000000,18.000000
25%,68.250000,67.000000,4.000000,157.250000
50%,135.500000,67.000000,4.200000,291.000000
75%,202.750000,169.500000,4.600000,572.750000
max,270.000000,235.000000,5.000000,3067.000000


Max tensor length can be 512 when doing sentiment analysis

Select which model to use for sentiment analysis

In [19]:
sentiment_pipeline = pipeline(model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

Device set to use mps:0


In [20]:
def add_sentiment_columns(df: pd.DataFrame, text_col: str = "review_text") -> pd.DataFrame:
    """
    Adds sentiment 'label' and 'score' columns to a DataFrame using a HuggingFace pipeline.
    If text_length > 512, only the first 512 characters are analyzed.

    Args:
        df (pd.DataFrame): Input DataFrame containing a column of text reviews.
        text_col (str): Name of the column with text to analyze.

    Returns:
        pd.DataFrame: Original DataFrame with new 'label' and 'score' columns.
    """

    def get_sentiment(text: str, length: int):
        if length > 512:
            text = text[:512]
        result = sentiment_pipeline(text)[0]
        return result["label"], result["score"]

    df[["label", "score"]] = df.apply(lambda row: get_sentiment(row[text_col], row["text_length"]), axis=1, result_type="expand")

    return df


In [21]:
add_sentiment_columns(df, text_col = "review")

,review_id,resort,full_day_price,avg_rating,review,text_length,label,score
0,1,Perisher Ski Resort,183.0,5.0,Highly recommended! Me and my family had a fan...,458,POSITIVE,0.999874
1,2,Perisher Ski Resort,183.0,5.0,"Visited Perisher valley, Smiggin Holes.. Which...",339,POSITIVE,0.991743
2,3,Perisher Ski Resort,183.0,3.0,Perisher Valley is absolutely magical – the sn...,871,NEGATIVE,0.989859
3,4,Perisher Ski Resort,183.0,2.0,We do the skiing once a year during the last 9...,1478,NEGATIVE,0.998968
4,5,Perisher Ski Resort,183.0,4.0,I think Perisher does a good job given the con...,194,NEGATIVE,0.997874
...,...,...,...,...,...,...,...,...
265,266,Selwyn,119.0,3.9,We visited on a Monday in school holidays and ...,775,POSITIVE,0.968631
266,267,Selwyn,119.0,3.9,Crowd management was poor!\n2 terminal out of ...,377,NEGATIVE,0.999516
267,268,Selwyn,119.0,3.9,"Small but amazing. Love the ""magic carpet"" for...",203,NEGATIVE,0.962833
268,269,Selwyn,119.0,3.9,Love this place! Perfect spot to learn to ski ...,237,POSITIVE,0.999855


In [23]:
# show number of each different value in 'label' column
label_counts = df['label'].value_counts()

print(label_counts)

label
POSITIVE    196
NEGATIVE     74
Name: count, dtype: int64


In [25]:
df.describe()

,review_id,full_day_price,avg_rating,text_length,score
count,270.000000,270.000000,270.000000,270.000000,270.000000
mean,135.500000,115.342593,4.242593,426.070370,0.975566
std,78.086491,61.129686,0.581819,426.651393,0.075614
min,1.000000,67.000000,1.000000,18.000000,0.514583
25%,68.250000,67.000000,4.000000,157.250000,0.994523
50%,135.500000,67.000000,4.200000,291.000000,0.999452
75%,202.750000,169.500000,4.600000,572.750000,0.999777
max,270.000000,235.000000,5.000000,3067.000000,0.999887


In [28]:
# find row where score is 0.514583
df_low_confidence = df[df['score'] < 0.52]

df_low_confidence.head()

,review_id,resort,full_day_price,avg_rating,review,text_length,label,score
172,173,Mount Hotham,67.0,4.7,Exceeded my expectation. Even a month felt les...,73,NEGATIVE,0.514583


That row is incorrect. Manually edit. 

In [33]:
# where review column row contains  "Exceeded my expectation. Even a month felt less. Can stay here all winter", change label value to POSITIVE and score to 1
df.loc[df['review'] == "Exceeded my expectation. Even a month felt less. Can stay here all winter", ['label', 'score']] = ['POSITIVE', 1]

df[df['review'] == "Exceeded my expectation. Even a month felt less. Can stay here all winter"]

,review_id,resort,full_day_price,avg_rating,review,text_length,label,score
172,173,Mount Hotham,67.0,4.7,Exceeded my expectation. Even a month felt les...,73,POSITIVE,1.0


Let's check other negative rows 

In [34]:
# create df for only NEGATIVE label
df_negative = df[df['label'] == 'NEGATIVE']

df_negative.head()

,review_id,resort,full_day_price,avg_rating,review,text_length,label,score
2,3,Perisher Ski Resort,183.0,3.0,Perisher Valley is absolutely magical – the sn...,871,NEGATIVE,0.989859
3,4,Perisher Ski Resort,183.0,2.0,We do the skiing once a year during the last 9...,1478,NEGATIVE,0.998968
4,5,Perisher Ski Resort,183.0,4.0,I think Perisher does a good job given the con...,194,NEGATIVE,0.997874
6,7,Perisher Ski Resort,183.0,1.0,My experience at Perisher was unfortunately th...,680,NEGATIVE,0.999798
10,11,Perisher Ski Resort,183.0,3.0,"I came to perisher to ski for the first time, ...",572,NEGATIVE,0.997934


In [ ]:
#export df as a csv file
# df.to_csv('resort_reviews_sentiment.csv', index=False)

Topic Modelling 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import copy

import nltk
from nltk.probability import *
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef, precision_recall_curve, auc


import gensim
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel

import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim_models

In [3]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/bryanho/nltk_data...


True

In [36]:
df_sentiment = pd.read_csv("resort_reviews_sentiment.csv")

In [37]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   review_id       270 non-null    int64  
 1   resort          270 non-null    object 
 2   full_day_price  270 non-null    float64
 3   avg_rating      270 non-null    float64
 4   review          270 non-null    object 
 5   text_length     270 non-null    int64  
 6   label           270 non-null    object 
 7   score           270 non-null    float64
dtypes: float64(3), int64(2), object(3)
memory usage: 17.0+ KB


In [38]:
df_sentiment.head()

,review_id,resort,full_day_price,avg_rating,review,text_length,label,score
0,1,Perisher Ski Resort,183.0,5.0,Highly recommended! Me and my family had a fan...,458,POSITIVE,0.999874
1,2,Perisher Ski Resort,183.0,5.0,"Visited Perisher valley, Smiggin Holes.. Which...",339,POSITIVE,0.991743
2,3,Perisher Ski Resort,183.0,3.0,Perisher Valley is absolutely magical – the sn...,871,NEGATIVE,0.989859
3,4,Perisher Ski Resort,183.0,2.0,We do the skiing once a year during the last 9...,1478,NEGATIVE,0.998968
4,5,Perisher Ski Resort,183.0,4.0,I think Perisher does a good job given the con...,194,NEGATIVE,0.997874


In [50]:
# for df_sentiment values do strip
df_sentiment['resort'] = df_sentiment['resort'].str.strip()

df_sentiment['resort'].unique()


array(['Perisher Ski Resort', 'Mt Buller', 'Mt Stirling',
       'Thredbo Alpine Hotel', 'Mount Baw Baw', 'Mount Hotham',
       'Charlotte Pass Snow Resort', 'Falls Creek', 'Selwyn'],
      dtype=object)

In [51]:
resort_names = list(df_sentiment.resort.unique())

separate resorts so each one has its own topic modelling

In [52]:
dfs_by_resort = {}

for resort in resort_names:
    dfs_by_resort[resort] = df_sentiment[df_sentiment['resort'] == resort]

In [53]:
list(dfs_by_resort.keys())

['Perisher Ski Resort',
 'Mt Buller',
 'Mt Stirling',
 'Thredbo Alpine Hotel',
 'Mount Baw Baw',
 'Mount Hotham',
 'Charlotte Pass Snow Resort',
 'Falls Creek',
 'Selwyn']

In [57]:
# turn or reviews into list of strings
resort_text_dict = {}

for resort, df in dfs_by_resort.items():
    reviews = df['review'].values.tolist()
    resort_text_dict[resort] = reviews

In [59]:
resort_text_dict['Mt Buller'][:3]

["Mt Buller is hands down one of the best ski resorts near Melbourne! I’ve been coming up every Sunday and it never disappoints — the slopes are well-maintained, and the overall vibe on the mountain is energetic and welcoming. My daughter has been taking private lessons, and her coach has been incredible — patient, knowledgeable, and genuinely invested in helping her improve. The progress she’s made has been amazing to watch. Whether you're a beginner or experienced skier, Mt Buller offers a fantastic snow experience for everyone. Highly recommend!",
 "Mount Buller in winter truly casts a spell! Even amidst a gentle mist, the pristine snow-covered landscapes and the crisp, invigorating mountain air weave an unforgettable tapestry of romance. From the shared thrills on the slopes to the cozy warmth by a crackling fire pit, and the breathtaking scenic chairlift rides offering intimate glimpses of the surrounding beauty, it's an absolutely incredible and deeply romantic alpine escape in V

Pre processing

In [61]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bryanho/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [62]:
stop_words  = nltk.corpus.stopwords.words('english')

lemmatizer = WordNetLemmatizer()

tokenizer = RegexpTokenizer(r'\w+')

['Perisher Ski Resort',
 'Mt Buller',
 'Mt Stirling',
 'Thredbo Alpine Hotel',
 'Mount Baw Baw',
 'Mount Hotham',
 'Charlotte Pass Snow Resort',
 'Falls Creek',
 'Selwyn']
 

In [65]:
corpus_perisher = resort_text_dict['Perisher Ski Resort']

corpus_buller = resort_text_dict['Mt Buller']

corpus_stirling = resort_text_dict['Mt Stirling']

corpus_thredbo = resort_text_dict['Thredbo Alpine Hotel']

corpus_baw_baw = resort_text_dict['Mount Baw Baw']

corpus_hotham = resort_text_dict['Mount Hotham']

corpus_charlotte = resort_text_dict['Charlotte Pass Snow Resort']

corpus_falls = resort_text_dict['Falls Creek']

corpus_selwyn = resort_text_dict['Selwyn']

In [67]:
def text_cleaner(docs):

    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()                     
        docs[idx] = tokenizer.tokenize(docs[idx])          

    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
    docs = [[token for token in doc if len(token) > 1] for doc in docs]
    docs = [[token for token in doc if token not in stop_words] for doc in docs]
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    return docs

In [68]:
corpus_perisher_cleaned = text_cleaner(copy.deepcopy(corpus_perisher))
corpus_buller_cleaned = text_cleaner(copy.deepcopy(corpus_buller))
corpus_stirling_cleaned = text_cleaner(copy.deepcopy(corpus_stirling))
corpus_thredbo_cleaned = text_cleaner(copy.deepcopy(corpus_thredbo))
corpus_baw_baw_cleaned = text_cleaner(copy.deepcopy(corpus_baw_baw))
corpus_hotham_cleaned = text_cleaner(copy.deepcopy(corpus_hotham))
corpus_charlotte_cleaned = text_cleaner(copy.deepcopy(corpus_charlotte))
corpus_falls_cleaned = text_cleaner(copy.deepcopy(corpus_falls))
corpus_selwyn_cleaned = text_cleaner(copy.deepcopy(corpus_selwyn))

In [70]:
def bigram_trigram_adder(docs, min_count= 20):
    # Add bigrams and trigrams to docs (only ones that appear 20 times or more).
    bigram = Phrases(docs, min_count=min_count)
    for idx in range(len(docs)):
        for token in bigram[docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                docs[idx].append(token)
    return bigram

In [71]:
corpus_perisher_bigram = bigram_trigram_adder(corpus_perisher_cleaned)
corpus_buller_bigram = bigram_trigram_adder(corpus_buller_cleaned)
corpus_stirling_bigram = bigram_trigram_adder(corpus_stirling_cleaned)
corpus_thredbo_bigram = bigram_trigram_adder(corpus_thredbo_cleaned)
corpus_baw_baw_bigram = bigram_trigram_adder(corpus_baw_baw_cleaned)
corpus_hotham_bigram = bigram_trigram_adder(corpus_hotham_cleaned)
corpus_charlotte_bigram = bigram_trigram_adder(corpus_charlotte_cleaned)
corpus_falls_bigram = bigram_trigram_adder(corpus_falls_cleaned)
corpus_selwyn_bigram = bigram_trigram_adder(corpus_selwyn_cleaned)


In [85]:
def doc_representation_filtered_dict(docs, no_below=2, no_above=0.7):
    # Create a dictionary representation of the documents.
    dictionary = Dictionary(docs)

    # Filter out words that occur less than 20 documents, or more than 50% of the documents.
    dictionary.filter_extremes(no_below=no_below, no_above=no_above)

    return dictionary

In [86]:
corpus_perisher_dict = doc_representation_filtered_dict(corpus_perisher_cleaned)
corpus_buller_dict = doc_representation_filtered_dict(corpus_buller_cleaned)
corpus_stirling_dict = doc_representation_filtered_dict(corpus_stirling_cleaned)
corpus_thredbo_dict = doc_representation_filtered_dict(corpus_thredbo_cleaned)
corpus_baw_baw_dict = doc_representation_filtered_dict(corpus_baw_baw_cleaned)
corpus_hotham_dict = doc_representation_filtered_dict(corpus_hotham_cleaned)
corpus_charlotte_dict = doc_representation_filtered_dict(corpus_charlotte_cleaned)
corpus_falls_dict = doc_representation_filtered_dict(corpus_falls_cleaned)
corpus_selwyn_dict = doc_representation_filtered_dict(corpus_selwyn_cleaned)


In [87]:
def bag_of_words_representation(docs, dictionary):
    # Create a bag of words representation of the documents.
    bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

    return bow_corpus

In [88]:
corpus_perisher_bow = bag_of_words_representation(corpus_perisher_cleaned, corpus_perisher_dict)
corpus_buller_bow = bag_of_words_representation(corpus_buller_cleaned, corpus_buller_dict)
corpus_stirling_bow = bag_of_words_representation(corpus_stirling_cleaned, corpus_stirling_dict)
corpus_thredbo_bow = bag_of_words_representation(corpus_thredbo_cleaned, corpus_thredbo_dict)
corpus_baw_baw_bow = bag_of_words_representation(corpus_baw_baw_cleaned, corpus_baw_baw_dict)
corpus_hotham_bow = bag_of_words_representation(corpus_hotham_cleaned, corpus_hotham_dict)
corpus_charlotte_bow = bag_of_words_representation(corpus_charlotte_cleaned, corpus_charlotte_dict)
corpus_falls_bow = bag_of_words_representation(corpus_falls_cleaned, corpus_falls_dict)
corpus_selwyn_bow = bag_of_words_representation(corpus_selwyn_cleaned, corpus_selwyn_dict)

Perisher

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_perisher_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_perisher_dict.id2token

corpus_perisher_mod = LdaModel(
    corpus=corpus_perisher_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_perisher_mod.save(outputfile)

Buller

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_buller_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_buller_dict.id2token

corpus_buller_mod = LdaModel(
    corpus=corpus_buller_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_buller_mod.save(outputfile)

Stirling

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_stirling_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_stirling_dict.id2token

corpus_stirling_mod = LdaModel(
    corpus=corpus_stirling_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_stirling_mod.save(outputfile)

Thredbo

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_thredbo_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_thredbo_dict.id2token

corpus_thredbo_mod = LdaModel(
    corpus=corpus_thredbo_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_thredbo_mod.save(outputfile)

Baw Baw

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_baw_baw_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_baw_baw_dict.id2token

corpus_baw_baw_mod = LdaModel(
    corpus=corpus_baw_baw_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_baw_baw_mod.save(outputfile)

Hotham

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_hotham_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_hotham_dict.id2token

corpus_hotham_mod = LdaModel(
    corpus=corpus_hotham_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_hotham_mod.save(outputfile)

Charlotte

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_charlotte_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_charlotte_dict.id2token

corpus_charlotte_mod = LdaModel(
    corpus=corpus_charlotte_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_charlotte_mod.save(outputfile)

Falls

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_falls_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_falls_dict.id2token

corpus_falls_mod = LdaModel(
    corpus=corpus_falls_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_falls_mod.save(outputfile)

Perisher

In [ ]:
# Set training parameters.
NUM_TOPICS = 10
chunksize = 1000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = corpus_selwyn_dict[0]  # This is only to "load" the dictionary.
id2word = corpus_selwyn_dict.id2token

corpus_selwyn_mod = LdaModel(
    corpus=corpus_selwyn_bow,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=NUM_TOPICS,
    passes=passes,
    eval_every=eval_every
)
outputfile = f'model{NUM_TOPICS}_1k.gensim'
print("Saving model in " + outputfile)
print("")
corpus_selwyn_mod.save(outputfile)

DONE
